In [ ]:
"""
"""

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# -------------------------Configuration Settings------------------------------------
# All tunable parameters are stored here for easy access.
CONFIG = {
    "file_path": "synthetic_fiscal_metering_data.xlsx",
    "features": ["Flow", "Pressure", "Temperature"],
    "timestamp_col": "Timestamp",
    "time_steps": 60,       # How many past data points the model sees to make a prediction.
    "train_split": 0.8,
    "lstm_units_1": 64,
    "lstm_units_2": 32,
    "dropout_rate": 0.2,    # Helps prevent overfitting.
    "learning_rate": 0.0005,
    "epochs": 50,
    "batch_size": 32,
    "output_dir": "Plots"
}

# Make sure the output directory exists.
if not os.path.exists(CONFIG["output_dir"]):
    os.makedirs(CONFIG["output_dir"])

# Set a professional plotting style.
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300


# ----------------------Data Handling Functions----------------------------------

def load_and_preprocess_data(config):
    """
    Loads the dataset from an Excel file, sets the timestamp as the index,
    and scales the features to a range of [0, 1].
    """
    print("Loading and preparing data...")
    try:
        df = pd.read_excel(config["file_path"])
    except FileNotFoundError:
        print(f"Error: The file '{config['file_path']}' was not found.")
        return None, None, None

    df[config["timestamp_col"]] = pd.to_datetime(df[config["timestamp_col"]])
    df = df.set_index(config["timestamp_col"])
    df.columns = config["features"]

    # Scaling is crucial for neural networks.
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    print("Data successfully loaded and scaled.")
    return scaled_data, scaler, df

def create_sequences(data, time_steps):
    """
    Converts the time-series data into time_steps, format required by LSTMs.
    """
    X, y = [], []
    for i in range(len(data) - time_steps):
        # The sequence of 'time_steps' length
        X.append(data[i:(i + time_steps)])
        # The value to be predicted
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

# -----------------Model Architecture-------------------------------------------

def build_lstm_model(input_shape, config):
    """
    Defines the stacked LSTM model architecture. We use two LSTM layers for
    capturing complex temporal patterns and Dropout for regularization.
    """
    print("Building the LSTM model...")
    model = Sequential([
        LSTM(config["lstm_units_1"], return_sequences=True, input_shape=input_shape),
        Dropout(config["dropout_rate"]),
        LSTM(config["lstm_units_2"], return_sequences=False),
        Dropout(config["dropout_rate"]),
        Dense(input_shape[1]) # The output layer has a neuron for each feature.
    ])

    optimizer = Adam(learning_rate=config["learning_rate"])
    model.compile(optimizer=optimizer, loss="mse")
    model.summary()
    return model


#----------------------------- Anomaly Detection Logic--------------------------

def detect_anomalies(model, X_test, y_test):
    """
    Uses the trained model to predict values on the test set, calculates the
    reconstruction error, and identifies anomalies using a statistical threshold.
    """
    print("Detecting anomalies on the test data...")
    y_pred = model.predict(X_test)

    # The "reconstruction error" is the difference between what the model expected and what actually happened.
    errors = np.mean(np.abs(y_test - y_pred), axis=1)

    # We'll use a dynamic threshold: mean error + 3 standard deviations.
    # This is a common heuristic for outlier detection.
    threshold = np.mean(errors) + 3 * np.std(errors)
    print(f"Calculated dynamic anomaly threshold: {threshold:.4f}")

    anomalies_mask = errors > threshold
    return y_pred, anomalies_mask, errors, threshold

# ----------------------Visualization/Plotting Functions--------------------------------

def plot_correlation_matrix(df, config):

    print("Creating feature correlation heatmap...")
    plt.figure(figsize=(10, 8))
    corr_matrix = df.corr()
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Feature Correlation Matrix', fontsize=16, weight='bold')
    plt.savefig(os.path.join(config["output_dir"], "00_correlation_matrix.png"))
    plt.close()
    print("Correlation matrix plot saved.")

def plot_prediction_error_distribution(errors, threshold, config):

    print("Plotting the distribution of prediction errors...")
    plt.figure(figsize=(10, 6))
    sns.histplot(errors, bins=50, kde=True, color='navy')
    plt.axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Anomaly Threshold ({threshold:.4f})')
    plt.title('Distribution of Model Prediction Error', fontsize=16, weight='bold')
    plt.xlabel('Mean Absolute Error', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(config["output_dir"], "prediction_error_distribution.png"))
    plt.close()
    print("Prediction error distribution plot saved.")

def plot_anomaly_results_parallel(full_df, y_test_rescaled, y_pred_rescaled,
                                  mask, feature_index, split_timestamp, config):
    """
    Creates a two-part plot for each feature: one showing the full time-series context
    and another on the test set to highlight detected anomalies.
    """
    feature_name = config["features"][feature_index]
    units = {"Flow": "Sm³/h", "Pressure": "bar", "Temperature": "°C"}

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=False)
    fig.suptitle(f'Analysis for {feature_name}', fontsize=20, weight='bold')

    # Top plot: The big picture
    ax1.plot(full_df.index, full_df[feature_name], color='navy', label='Full Raw Data')
    ax1.axvline(x=split_timestamp, color='red', linestyle='--', linewidth=2, label='Train/Test Split')
    ax1.set_title(f'Full Time-Series Data for {feature_name}', fontsize=16)
    ax1.set_ylabel(f'{feature_name} ({units.get(feature_name, "")})', fontsize=12)
    ax1.legend()
    ax1.grid(True)

    # Bottom plot: Zoomed-in view of the test set results
    test_indices = full_df.index[-len(y_test_rescaled):]
    ax2.plot(test_indices, y_test_rescaled[:, feature_index], label=f'Actual {feature_name}', color='navy', alpha=0.8)
    ax2.plot(test_indices, y_pred_rescaled[:, feature_index], label=f'Predicted {feature_name}', color='darkorange', linestyle='--')

    # Highlight the anomalies we found
    anomalous_indices = np.where(mask)[0]
    ax2.scatter(
        test_indices[anomalous_indices],
        y_test_rescaled[anomalous_indices, feature_index],
        color='red', marker='o', s=50, label='Detected Anomaly'
    )

    ax2.set_title('Anomaly Detection Results on Test Set', fontsize=16)
    ax2.set_xlabel('Timestamp', fontsize=12)
    ax2.set_ylabel(f'{feature_name} ({units.get(feature_name, "")})', fontsize=12)
    ax2.legend()
    ax2.grid(True)
    plt.xticks(rotation=45)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    output_path = os.path.join(config["output_dir"], f"04_{feature_name}_analysis.png")
    plt.savefig(output_path)
    plt.close()
    print(f"Parallel analysis plot for {feature_name} saved.")


# -----------------------Main Execution------------------------------------------

def main():
    """Runs the full anomaly detection pipeline from start to finish."""

    # --- 1. Data Loading and Preparation ---
    scaled_data, scaler, full_df = load_and_preprocess_data(CONFIG)
    if full_df is None:
        return # Stop execution if data loading failed

    # Let's see how our features are related before we start modeling.
    plot_correlation_matrix(full_df, CONFIG)

    X, y = create_sequences(scaled_data, CONFIG["time_steps"])
    split_idx = int(CONFIG["train_split"] * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    print(f"Data split into {len(X_train)} training and {len(X_test)} testing samples.")

    # Find the exact timestamp of the split for plotting later.
    split_timestamp = full_df.index[split_idx + CONFIG["time_steps"]]

    # --- 2. Model Training ---
    model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]), config=CONFIG)
    print("\nKicking off model training. This might take a moment...")
    model.fit(
        X_train, y_train,
        epochs=CONFIG["epochs"],
        batch_size=CONFIG["batch_size"],
        validation_split=0.2,
        shuffle=False,
        verbose=1
    )
    print("Model training is complete.")

    # --- 3. Anomaly Detection & Performance Evaluation ---
    y_pred, anomalies_mask, errors, threshold = detect_anomalies(model, X_test, y_test)

    # Calculate Mean Squared Error on the scaled test data.
    mse = mean_squared_error(y_test, y_pred)
    print(f"Prediction Mean Squared Error (MSE): {mse:.6f}")

    # Since we don't have ground-truth anomaly labels, we'll assume all test points
    # are 'normal' to evaluate how many false positives our model generated.
    y_true_labels = np.zeros(len(y_test))
    y_pred_labels = anomalies_mask.astype(int)


    # Visualize the error distribution to see if the threshold makes sense.
    plot_prediction_error_distribution(errors, threshold, CONFIG)

    # --- 4. Generate Final Result Plots ---
    print("\nGenerating final analysis plots for each feature...")
    # Rescale data back to original units for interpretability.
    y_test_rescaled = scaler.inverse_transform(y_test)
    y_pred_rescaled = scaler.inverse_transform(y_pred)

    for i in range(len(CONFIG["features"])):
        plot_anomaly_results_parallel(
            full_df, y_test_rescaled, y_pred_rescaled,
            anomalies_mask, feature_index=i,
            split_timestamp=split_timestamp, config=CONFIG
        )


if __name__ == "__main__":
    main()